# Migrate Mongo data from `nmdc-schema` [`v8.0.0`](https://github.com/microbiomedata/nmdc-schema/releases/tag/v8.0.0) to [`v8.1.2`](https://github.com/microbiomedata/nmdc-schema/releases/tag/v8.1.2)

## Prerequisites

### 1. Determine Mongo collections that will be transformed

In this step, you will determine which Mongo collections will be transformed during this migration.

1. In [`nmdc_schema/migration_recursion.py`](https://github.com/microbiomedata/nmdc-schema/blob/main/nmdc_schema/migration_recursion.py), locate the Python class whose name reflects the initial and final version numbers of this migration.
2. In that Python class, locate the `self.agenda` dictionary.
3. In that dictionary, make a list of the keys—these are the names of the Mongo collections that will be transformed during this migration. For example:
   ```py
   self.agenda = dict(
      collection_name_1=[self.some_function],
      collection_name_2=[self.some_function],
   )
   ```

### 2. Coordinate with teammates that read/write to those collections

In this step, you'll identify and reach out to the people that read/write to those collections; to agree on a migration schedule that works for you and them.

Here's a table of Mongo collections and the components of the NMDC system that write to them (according to [a conversation that occurred on September 11, 2023](https://nmdc-group.slack.com/archives/C01SVTKM8GK/p1694465755802979?thread_ts=1694216327.234519&cid=C01SVTKM8GK)).

| Mongo collection                            | NMDC system components that write to it                  |
|---------------------------------------------|----------------------------------------------------------|
| `biosample_set`                             | Workflows (via manual entry via `nmdc-runtime` HTTP API) |
| `data_object_set`                           | Workflows (via `nmdc-runtime` HTTP API)                  |
| `mags_activity_set`                         | Workflows (via `nmdc-runtime` HTTP API)                  |
| `metagenome_annotation_activity_set`        | Workflows (via `nmdc-runtime` HTTP API)                  |
| `metagenome_assembly_set`                   | Workflows (via `nmdc-runtime` HTTP API)                  |
| `read_based_taxonomy_analysis_activity_set` | Workflows (via `nmdc-runtime` HTTP API)                  |
| `read_qc_analysis_activity_set`             | Workflows (via `nmdc-runtime` HTTP API)                  |
| `jobs`                                      | Scheduler (via Mongo directly)                           |
| `*`                                         | `nmdc-runtime` (via Mongo directly)                      |

You can use that table to help determine which people read/write to those collections. You can then coordinate a migration time slot with them via Slack, email, etc.

### 3. Setup a migration environment

In this step, you'll set up an environment in which you can run this notebook.

1. Start a **Mongo server** on your local machine (and ensure it does **not** contain a database named `nmdc`).
    1. You can start a temporary, [Docker](https://hub.docker.com/_/mongo)-based Mongo server at `localhost:27055` by running this command:
       ```shell
       docker run --rm --detach --name mongo-migration-transformer -p 27055:27017 mongo
       ```
       > Note: A Mongo server started via that command will have no access control (i.e. you will be able to access it without a username or password).
2. Create and populate a **notebook configuration file** named `.notebook.env`.
    1. You can use the `.notebook.env.example` file as a template:
       ```shell
       $ cp .notebook.env.example .notebook.env
       ```
3. Create and populate **Mongo configuration files** for connecting to the origin and transformer Mongo servers.
    1. You can use the `.mongo.yaml.example` file as a template:
       ```shell
       $ cp .mongo.yaml.example .mongo.origin.yaml
       $ cp .mongo.yaml.example .mongo.transformer.yaml
       ```
       > When populating the file for the origin Mongo server, use credentials that have write access to the `nmdc` database.

## Procedure

### Install Python dependencies

In this step, you'll [install](https://saturncloud.io/blog/what-is-the-difference-between-and-in-jupyter-notebooks/) the Python packages upon which this notebook depends. You can do that by running this cell.

> Note: If the output of this cell says "Note: you may need to restart the kernel to use updated packages", restart the kernel (not the notebook) now.

In [ ]:
%pip install -r requirements.txt
%pip install nmdc-schema==8.1.2

### Import Python dependencies

Import the Python objects upon which this notebook depends.

> Note: One of the Python objects is a Python class that is specific to this migration.

In [ ]:
# Standard library packages:
from pathlib import Path
from shutil import rmtree
from copy import deepcopy

# Third-party packages:
import pymongo
from nmdc_schema.nmdc_data import get_nmdc_jsonschema_dict
from nmdc_schema.migration_recursion import Migrator_from_8_0_0_to_8_1_0 as Migrator
from jsonschema import Draft7Validator
from dictdiffer import diff

# First-party packages:
from helpers import Config

### Programmatically determine which collections will be transformed

Here are the names of the collections this migration will transform.

> Ensure you have coordinated with the people that read/write to them.

In [ ]:
agenda_collection_names = Migrator().agenda.keys()

print("The following collections will be transformed:")
print("\n".join(agenda_collection_names))

### Parse configuration files

Parse the notebook and Mongo configuration files.

In [ ]:
cfg = Config()

# Define some aliases we can use to make the shell commands in this notebook easier to read.
mongodump = cfg.mongodump_path
mongorestore = cfg.mongorestore_path

Perform a sanity test of the application paths.

In [ ]:
!{mongodump} --version
!{mongorestore} --version

### Create Mongo clients

Create Mongo clients you can use to access the "origin" Mongo server (i.e. the one containing the database you want to migrate) and the "transformer" Mongo server (i.e. the one you want to use to perform the data transformations).

In [ ]:
# Mongo client for origin Mongo server.
origin_mongo_client = pymongo.MongoClient(host=cfg.origin_mongo_server_uri, directConnection=True)

# Mongo client for transformer Mongo server.
transformer_mongo_client = pymongo.MongoClient(host=cfg.transformer_mongo_server_uri)

Perform a sanity test of the Mongo clients' abilities to access their respective Mongo servers.

In [ ]:
# Display the Mongo server version (running on the "origin" Mongo server).
print("Origin Mongo server version:      " + origin_mongo_client.server_info()["version"])

# Sanity test: Ensure the origin database exists.
assert "nmdc" in origin_mongo_client.list_database_names(), "Origin database does not exist."

# Display the Mongo server version (running on the "transformer" Mongo server).
print("Transformer Mongo server version: " + transformer_mongo_client.server_info()["version"])

# Sanity test: Ensure the transformation database does not exist.
assert "nmdc" not in transformer_mongo_client.list_database_names(), "Transformation database already exists."

### Create JSON Schema validator

In this step, you'll create a JSON Schema validator for the NMDC Schema.

In [ ]:
nmdc_jsonschema: dict = get_nmdc_jsonschema_dict()
nmdc_jsonschema_validator = Draft7Validator(nmdc_jsonschema)

Perform sanity tests of the NMDC Schema dictionary and the JSON Schema validator.

> Reference: https://python-jsonschema.readthedocs.io/en/latest/api/jsonschema/protocols/#jsonschema.protocols.Validator.check_schema

In [ ]:
print("NMDC Schema title:   " + nmdc_jsonschema["title"])
print("NMDC Schema version: " + nmdc_jsonschema["version"])

nmdc_jsonschema_validator.check_schema(nmdc_jsonschema)  # raises exception if schema is invalid

### Dump collections from the "origin" Mongo server

In this step, you'll use `mongodump` to dump the collections that will be transformed during this migration; from the "origin" Mongo server.

Since `mongodump` doesn't provide a CLI option that you can use to specify the collections you _want_ it to dump (unless that is only one collection), you can use a different CLI option to tell it all the collection you do _not_ want it to dump. The end result will be the same—there's just an extra step involved.

That extra step is to generate an `--excludeCollection="{name}"` CLI option for each collection that is not on the agenda, which you'll do now.

In [ ]:
# Build a string containing zero or more `--excludeCollection="..."` options, which can be included in a `mongodump` command.
all_collection_names: list[str] = origin_mongo_client["nmdc"].list_collection_names()
non_agenda_collection_names = [name for name in all_collection_names if name not in agenda_collection_names]
exclusion_options = [f"--excludeCollection='{name}'" for name in non_agenda_collection_names]
exclusion_options_str = " ".join(exclusion_options)  # separates each option with a space

print(exclusion_options_str)

Here, you'll run a `mongodump` command containing all those `--excludeCollection="{name}"` CLI options.

In [ ]:
# Dump the not-excluded collections from the origin database.
!{mongodump} \
  --config="{cfg.origin_mongo_config_file_path}" \
  --db="nmdc" \
  --gzip \
  --out="{cfg.origin_dump_folder_path}" \
  {exclusion_options_str}

### Load the collections into the "transformer" Mongo server

In this step, you'll load the collections dumped from the "origin" Mongo server, into the "transformer" MongoDB server.

Since it's possible that the dump includes more collections than are on the agenda (due to someone creating a collection between the time you generated the exclusion list and the time you ran `mongodump`), you will use one or more of `mongorestore`'s `--nsInclude` CLI options to indicate which collections you want to load.

Here's where you will generate the `--nsInclude="nmdc.{name}"` CLI options.

In [ ]:
inclusion_options = [f"--nsInclude='nmdc.{name}'" for name in agenda_collection_names]
inclusion_options_str = " ".join(inclusion_options)  # separates each option with a space

print(inclusion_options_str)

Here, you'll run a `mongorestore` command containing all those `--nsInclude="nmdc.{name}"` CLI options.

In [ ]:
# Restore the dumped collections to the transformer MongoDB server.
!{mongorestore} \
  --config="{cfg.transformer_mongo_config_file_path}" \
  --gzip \
  --drop \
  --preserveUUID \
  --dir="{cfg.origin_dump_folder_path}" \
  {inclusion_options_str}

### Transform the collections within the "transformer" Mongo server

Now that the transformer database contains a copy of each collection on the agenda, you can transform those copies.

The transformation functions are provided by the `nmdc-schema` Python package.
> You can examine the transformation functions at: https://github.com/microbiomedata/nmdc-schema/blob/main/nmdc_schema/migration_recursion.py

In this step, you will retrieve each documents from each collection on the agenda, pass it to the associated transformation function(s) on the agenda, then store the transformed document in place of the original one—all within the "transformation" database only. **The "origin" database is not involved with this step.**

> Note: This step also includes validation. Reference: https://github.com/microbiomedata/nmdc-runtime/blob/main/metadata-translation/src/bin/validate_json.py

> Note: This step also include a before-and-after comparison to facilitate manual spot checks. References: https://docs.python.org/3/library/copy.html#copy.deepcopy and https://dictdiffer.readthedocs.io/

In [ ]:
migrator = Migrator()

# Apply the transformations.
for collection_name, transformation_pipeline in migrator.agenda.items():
    print(f"Transforming documents in collection: {collection_name}")
    transformed_documents = []

    # Get each document from this collection.
    collection = transformer_mongo_client["nmdc"][collection_name]
    for original_document in collection.find():
        # Make a deep copy of the original document, to enable before-and-after comparison.
        print(original_document)
        copy_of_original_document = deepcopy(original_document)
        
        # Put the document through the transformation pipeline associated with this collection.
        transformed_document = original_document  # initializes the variable
        for transformation_function in transformation_pipeline:
            transformed_document = transformation_function(transformed_document)
        print(transformed_document)
        
        # Compare the transformed document with a copy of the original document;
        # and, if there are any differences, print those differences.
        difference = diff(copy_of_original_document, transformed_document)
        differences = list(difference)
        if len(differences) > 0:
            print(f"✏️ {differences}")

        # Validate the transformed document.
        #
        # Reference: https://github.com/microbiomedata/nmdc-schema/blob/main/src/docs/schema-validation.md
        #
        # Note: Dictionaries originating as Mongo documents include a Mongo-generated key named `_id`. However,
        #       the NMDC Schema does not describe that key and, indeed, data validators consider dictionaries
        #       containing that key to be invalid with respect to the NMDC Schema. So, here, we validate a
        #       copy (i.e. a shallow copy) of the document that lacks that specific key.
        #
        # Note: `root_to_validate` is a dictionary having the shape: { "some_collection_name": [ some_document ] }
        #       Reference: https://docs.python.org/3/library/stdtypes.html#dict (see the "type constructor" section)
        #
        transformed_document_without_underscore_id_key = {key: value for key, value in transformed_document.items() if key != "_id"}
        root_to_validate = dict([(collection_name, [transformed_document_without_underscore_id_key])])
        nmdc_jsonschema_validator.validate(root_to_validate)  # raises exception if invalid

        # Store the transformed document.
        transformed_documents.append(transformed_document)    
        print("")    

    # Replace the original documents with the transformed versions of themselves (in the transformer database).
    for transformed_document in transformed_documents:
        collection.replace_one({"id": {"$eq": transformed_document["id"]}}, transformed_document)


### Dump the transformed collections

In [ ]:
# Dump the database from the transformer MongoDB server.
!{mongodump} \
  --config="{cfg.transformer_mongo_config_file_path}" \
  --db="nmdc" \
  --gzip \
  --out="{cfg.transformer_dump_folder_path}" \
  {exclusion_options_str}

### Load the transformed data into the "origin" Mongo server

In this step, you'll put the transformed collection(s) into the origin MongoDB server, replacing the original collection(s) that have the same name(s).

In [ ]:
# Replace the same-named collection(s) on the origin server, with the transformed one(s).
!{mongorestore} \
  --config="{cfg.origin_mongo_config_file_path}" \
  --gzip \
  --verbose \
  --dir="{cfg.transformer_dump_folder_path}" \
  --drop \
  --preserveUUID \ 
  {inclusion_options_str}

### (Optional) Clean up

Delete the temporary files and MongoDB dumps created by this notebook.

> Note: You can skip this step, in case you want to delete them manually later (e.g. to examine them before deleting them).

In [ ]:
paths_to_files_to_delete = []

paths_to_folders_to_delete = [
    cfg.origin_dump_folder_path,
    cfg.transformer_dump_folder_path,
]

# Delete files.
for path in [Path(string) for string in paths_to_files_to_delete]:
    try:
        path.unlink()
        print(f"Deleted: {path}")
    except:
        print(f"Failed to delete: {path}")

# Delete folders.
for path in [Path(string) for string in paths_to_folders_to_delete]:
    try:
        rmtree(path)
        print(f"Deleted: {path}")
    except:
        print(f"Failed to delete: {path}")